# Semantic Search

## Import Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from DLAIUtils import Utils
import DLAIUtils
import sys

import os
import time
import torch

In [3]:
from tqdm.auto import tqdm

## Load Dataset

In [9]:
dataset = load_dataset('quora', split='train[240000:290000]', trust_remote_code=True)


Generating train split: 100%|██████████| 404290/404290 [00:05<00:00, 75713.81 examples/s]


In [10]:
dataset[:5]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']},
  {'id': [37799, 94186],
   'text': ['What is a good way to lose 30 pounds in 2 months?',
    'What can I do to lose 30 pounds in 2 months?']},
  {'id': [351733, 351734],
   'text': ['Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?',
    'How do you graph x + 2y = -2?']}],
 'is_duplicate': [False, True, False, True, False]}

In [11]:
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
question = list(set(questions))
print('\n'.join(questions[:10]))
print('-' * 50)
print(f'Number of questions: {len(questions)}')

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
Can I send homemade herbal hair oil from India to US via postal or private courier services?
What is a good way to lose 30 pounds in 2 months?
What can I do to lose 30 pounds in 2 months?
Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?
How do you graph x + 2y = -2?
--------------------------------------------------
Number of questions: 100000


## Check Cuda and model setup

Note: "Checking cuda" refers to checking if you have access to GPUs (faster compute). 

I am using all-MiniLM-L6-v2 sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry no cuda.


In [13]:
query = 'which city is the most populated in the world?'
xq = model.encode(query)
xq.shape

(384,)

## Setup Pinecone

In [21]:
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()

In [26]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = utils.create_dlai_index_name('dl-ai')

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)
pinecone.create_index(name=INDEX_NAME, 
    dimension=model.get_sentence_embedding_dimension(), 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)
print(index)

dl-ai-ylfg85fiye1wn9c-k01oijx-uzklzn5jp2oa


## Create Embeddings and Upsert to Pinecone

In [27]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    # create embeddings
    xc = model.encode(questions[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

100%|██████████| 50/50 [00:44<00:00,  1.12it/s]


In [28]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 10200}},
 'total_vector_count': 10200,
 'vector_type': 'dense'}

## Query Run

In [29]:
# small helper function so we can repeat queries later
def run_query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
  for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

In [30]:
run_query('which city has the highest population in the world?')

0.67: Which is the most urbanised city in India?
0.64: Which is the largest country in the world?
0.64: Which city has the most museums per capita?
0.61: How's the world's population determined?
0.56: What are the largest cities in Spain?
0.56: Why does China and several other Asian countries have such a high population?
0.55: Which is best city in India?
0.54: What are the largest slums in the world?
0.52: What are the best cities in india?
0.5: What is the most dangerous US city? Why is it considered so dangerous?


In [31]:
query = 'how do i make chocolate cake?'
run_query(query)

0.79: How do you add chocolate chips to cake mix?
0.57: How do you make pancakes?
0.55: How do you make baking soda?
0.52: How do you make shepherd's pie?
0.51: How do we bake cake in microwave oven? What should be the perfect temperature?
0.5: What does red velvet cake taste like?
0.49: What is the difference between chocolate and truffles?
0.46: How can I make homemade pancakes without baking soda?
0.45: Why is banana bread considered a bread and not a cake?
0.43: Where can I get highest quality, tastiest cupcakes across the Gold Coast?
